In [15]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import selenium
from datetime import datetime
from datetime import timedelta
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import csv


def xpath(row=0,col=0,sub=0,header=False):
    if header == True:
        xp = '//app/city-history/city-history-layout/div/div[2]/'\
            + 'section/div[2]/div[3]/div/div/div/div/'\
            + 'city-history-observation/div/div[2]/table'\
            + '/thead/tr/td['+str(col+1)+']'
    else:
        xp = '//app/city-history/city-history-layout/div/div[2]/'\
            + 'section/div[2]/div[3]/div/div/div/div/'\
            + 'city-history-observation/div/div[2]/table'\
            + '/tbody/tr/td[' + str(col+1) + ']'\
            + '/table/tbody/tr[' + str(row+1) + ']'\
            + '/td[' + str(sub+1) + ']'
    return xp



def read_webpage(driver,url):
    # Record start time (for scraper performance purposes)
    start_time = datetime.now()
    
    driver.get(url)

    time.sleep(3)
    
    webpage_data = []

    row_idx = 0
    column_idx = 0
    row_data = []
    while True:
        # For each column,
        # ...initialize subcolumn_idx to 0 and column_data to [], and ...
        subcolumn_idx = 0
        column_data = []
        try:
            # ...try to read the subcolumn at the following xpath to see if the column exists.
            raw_main_header = driver.find_element_by_xpath(xpath(col=column_idx, header=True))
            # If that worked, then iterate through subcolumns.
            while True:
                # For each subcolumn,
                try:
                    # ...try to read the subcolumn at the following xpath to see if it exists.
                    raw_sub_header = driver.find_element_by_xpath(xpath(row=row_idx,col=column_idx,sub=subcolumn_idx))
                    header = raw_sub_header.text+' '+raw_main_header.text
                    # If that worked, then store the concatenation of sub_header and main_header in column_data.
                    column_data.append(header)
                except:
                    # If it didn't work, there is no subcolumn at that index. Break subcolumn loop.
                    break

                # next subcolumn
                subcolumn_idx += 1

            # That's it for the subcolumns in this column. Store column_data in row_data.
            row_data.extend(column_data)

        except:
            # If it didn't work, there is no column at that index. Break column loop.
            break

        # next column
        column_idx += 1

    # That's it for the columns in this row.  Store row_data in webpage_data.
    webpage_data.append(row_data)
    

    # rows
    row_idx = 1
    column_idx = 0
    subcolumn_idx = 0

    # Iterate through rows.
    while True:
        # For each row,
        # ...initialize column_idx to 0 and row_data to [], and ...
        column_idx = 0
        row_data = []
                
        try:
            # ...try to read the subcolumn at the following xpath to see if the row exists.
            raw_datum = driver.find_element_by_xpath(xpath(row=row_idx,col=column_idx,sub=subcolumn_idx))

            # If that worked, then iterate through columns.
            while True:
                # For each column,
                # ...initialize subcolumn_idx to 0 and column_data to [], and ...
                subcolumn_idx = 0
                column_data = []
                
                try:
                    # ...try to read the subcolumn at the following xpath to see if the column exists.
                    raw_datum = driver.find_element_by_xpath(xpath(row=row_idx,col=column_idx,sub=subcolumn_idx))

                    # If that worked, then iterate through subcolumns
                    while True:
                        # For each subcolumn,
                        try:
                            # ...try to read the subcolumn at the following xpath to see if it exists.
                            raw_datum = driver.find_element_by_xpath(xpath(row=row_idx,col=column_idx,sub=subcolumn_idx))

                            # If that worked, then try to convert raw_datum.text to a float.
                            try:
                                if column_idx == 0:
                                    datum = int(raw_datum.text) # Store dates as integers.
                                else:
                                    datum = float(raw_datum.text) # Store measured values as floats.
                            except:
                                datum = float('nan') # Store dashes and anything that doesn't cooperate as NaNs.
                            column_data.append(datum)
                        except:
                            # If it didn't work, there is no subcolumn at that index. Break subcolumn loop.
                            break

                        # next subcolumn
                        subcolumn_idx += 1

                    # That's it for the subcolumns in this column. Store column_data in row_data.
                    row_data.extend(column_data)

                except:
                    #If it didn't work, there is no column at that index. 
                    break

                # next column
                column_idx += 1

            # That's it for the columns in this row.  Store row_data in webpage_data.
            webpage_data.append(row_data)
            
        except:
            # If it didn't work, there is no row at that index.
            break

        # next row
        row_idx += 1

    # That's it for the rows in the table. Done with this webpage!

    stop_time = datetime.now()
    time_diff = stop_time-start_time

    return webpage_data, time_diff


chromedriver = '/Users/grazillionaire/Documents/chromedriver' # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)


# Generate list of dates (months) to pull weather data for.

# For Chicago
city = 'il/chicago/KORD'
start_date = datetime(2001,1,1)
end_date = datetime(2018,6,30)

# For New York
#city = 'ny/new-york/KNYC'
#start_date = datetime(2010,5,5)
#end_date = datetime.today()

date_list = pd.Series(pd.date_range(start_date,end_date))
date_list = [datetime.today()] ################# delete this ##################
#city = 'ca/san-francisco/KSFO' ################ delete this ##################




for date in date_list:
    
    url = 'https://www.wunderground.com/history/monthly/us/{loc}/date/{y}-{m}?cm_ven=localwx_history'\
          .format(loc=city, y=date.year, m=date.month)
    
    webpage_data, elapsed = read_webpage(driver, url)

    print('Chicago weather data for ',date.month,'/',date.year,' scraped in ', elapsed.seconds + elapsed.microseconds/1000000, 's')


    myFile = open('CHI weather data/CHI-{y}-{m}.csv'.format(y=date.year, m=date.month), 'w')
    with myFile:
        writer = csv.writer(myFile)
        writer.writerows(webpage_data)

driver.close()

Chicago weather data for  10 / 2018  scraped in  166.474797 s


In [14]:
for row in webpage_data:
    print(row)
    #print([webpage_data[0][0][:3]+' '+str(row[0])]+row[1:])

['Oct Time', 'Max Temperature (° F)', 'Avg Temperature (° F)', 'Min Temperature (° F)', 'Max Dew Point (° F)', 'Avg Dew Point (° F)', 'Min Dew Point (° F)', 'Max Humidity (%)', 'Avg Humidity (%)', 'Min Humidity (%)', 'Max Wind Speed (mph)', 'Avg Wind Speed (mph)', 'Min Wind Speed (mph)', 'Max Pressure (Hg)', 'Avg Pressure (Hg)', 'Min Pressure (Hg)', 'Max Precipation (in)', 'Avg Precipation (in)', 'Min Precipation (in)']
[1, 72.0, 65.0, 57.0, 64.0, 58.0, 54.0, 93.0, nan, 68.0, 18.0, nan, 0.0, 30.23, nan, 29.97, nan, 1.56, nan]
[2, 67.0, 63.0, 58.0, 65.0, 57.0, 54.0, 93.0, nan, 72.0, 15.0, nan, 0.0, 30.11, nan, 29.97, nan, 0.01, nan]
[3, 88.0, 74.0, 59.0, 68.0, 62.0, 56.0, 93.0, nan, 45.0, 28.0, nan, 6.0, 30.05, nan, 29.68, nan, 0.0, nan]
[4, 77.0, 65.0, 52.0, 69.0, 51.0, 41.0, 93.0, nan, 61.0, 24.0, nan, 9.0, 30.22, nan, 29.75, nan, 0.04, nan]
[5, 65.0, 60.0, 54.0, 63.0, 57.0, 45.0, 93.0, nan, 62.0, 16.0, nan, 0.0, 30.12, nan, 29.91, nan, 1.24, nan]
[6, 70.0, 64.0, 57.0, 66.0, 59.0, 50.

In [17]:
subcolumn_idx

3

In [23]:
xpath(3,4,5)

'//app/city-history/city-history-layout/div/div[2]/section/div[2]/div[3]/div/div/div/div/city-history-observation/div/div[2]/table/tbody/tr/td[5]/table/tbody/tr[4]/td[6]'

In [25]:
xpath(col=10)

'//app/city-history/city-history-layout/div/div[2]/section/div[2]/div[3]/div/div/div/div/city-history-observation/div/div[2]/table/tbody/tr/td[11]/table/tbody/tr[1]/td[1]'